# Setup

In [ ]:
from GE2PE import GE2PE

g2p = GE2PE(model_path='GE2PE/phase3-t5/checkpoint-487100', GPU=True)


/media/external_10TB/mahta_fetrat/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
import re
from jiwer import cer
from difflib import SequenceMatcher

# Setup POSTagger

In [3]:
# ! git clone https://huggingface.co/roshan-research/spacy_pos_tagger_parsbertpostagger

In [4]:
import spacy
# spacy.require_gpu()
"""این ماژول شامل کلاس‌ها و توابعی برای برچسب‌گذاری توکن‌هاست."""

from nltk.tag import stanford  # noqa: I001
from hazm import SequenceTagger

import os
import subprocess

from spacy.tokens import Doc
from spacy.tokens import DocBin
from spacy.vocab import Vocab

from sklearn.metrics import classification_report,f1_score,accuracy_score,precision_score,recall_score

from tqdm import tqdm


punctuation_list = [
    '"',
    "#",
    "(",
    ")",
    "*",
    ",",
    "-",
    ".",
    "/",
    ":",
    "[",
    "]",
    "«",
    "»",
    "،",
    ";",
    "?",
    "!",
]


class POSTagger(SequenceTagger):
    """این کلاس‌ها شامل توابعی برای برچسب‌گذاری توکن‌هاست."""

    def __init__(
        self: "POSTagger", model=None, data_maker=None, universal_tag=False,
    ) -> None:
        data_maker = self.data_maker if data_maker is None else data_maker
        self.__is_universal = universal_tag
        super().__init__(model, data_maker)

    def __universal_converter(self: "POSTagger", tagged_list):
        return [(word, tag.split(",")[0]) for word, tag in tagged_list]

    def __is_punc(self: "POSTagger", word):
        return word in punctuation_list

    def data_maker(self: "POSTagger", tokens):
        """تابعی که لیستی از لیستی از کلمات توکنایز شده را گرفته و لیست دو بعدی از از دیکشنری‌هایی که تعیین‌کننده ویژگی‌ها هر کلمه هستند را برمی‌گرداند.

        Examples:
            >>> posTagger = POSTagger(model = 'pos_tagger.model')
            >>> posTagger.data_maker(tokens = [['دلم', 'اینجا', 'مانده‌است', '.']])
            [[{'word': 'دلم', 'is_first': True, 'is_last': False, 'prefix-1': 'د', 'prefix-2': 'دل', 'prefix-3': 'دلم', 'suffix-1': 'م', 'suffix-2': 'لم', 'suffix-3': 'دلم', 'prev_word': '', 'two_prev_word': '', 'next_word': 'اینجا', 'two_next_word': 'مانده\u200cاست', 'is_numeric': False, 'prev_is_numeric': '', 'next_is_numeric': False, 'is_punc': False, 'prev_is_punc': '', 'next_is_punc': False}, {'word': 'اینجا', 'is_first': False, 'is_last': False, 'prefix-1': 'ا', 'prefix-2': 'ای', 'prefix-3': 'این', 'suffix-1': 'ا', 'suffix-2': 'جا', 'suffix-3': 'نجا', 'prev_word': 'دلم', 'two_prev_word': '.', 'next_word': 'مانده\u200cاست', 'two_next_word': '.', 'is_numeric': False, 'prev_is_numeric': False, 'next_is_numeric': False, 'is_punc': False, 'prev_is_punc': False, 'next_is_punc': False}, {'word': 'مانده\u200cاست', 'is_first': False, 'is_last': False, 'prefix-1': 'م', 'prefix-2': 'ما', 'prefix-3': 'مان', 'suffix-1': 'ت', 'suffix-2': 'ست', 'suffix-3': 'است', 'prev_word': 'اینجا', 'two_prev_word': 'دلم', 'next_word': '.', 'two_next_word': '', 'is_numeric': False, 'prev_is_numeric': False, 'next_is_numeric': False, 'is_punc': False, 'prev_is_punc': False, 'next_is_punc': True}, {'word': '.', 'is_first': False, 'is_last': True, 'prefix-1': '.', 'prefix-2': '.', 'prefix-3': '.', 'suffix-1': '.', 'suffix-2': '.', 'suffix-3': '.', 'prev_word': 'مانده\u200cاست', 'two_prev_word': 'اینجا', 'next_word': '', 'two_next_word': '', 'is_numeric': False, 'prev_is_numeric': False, 'next_is_numeric': '', 'is_punc': True, 'prev_is_punc': False, 'next_is_punc': ''}]]

        Args:
            tokens (List[List[str]]): جملاتی که نیاز به تبدیل آن به برداری از ویژگی‌ها است.

        Returns:
            List(List(Dict())): لیستی از لیستی از دیکشنری‌های بیان‌کننده ویژگی‌های یک کلمه.
        """
        return [
            [self.features(token, index) for index in range(len(token))]
            for token in tokens
        ]

    def features(self: "POSTagger", sentence, index):
        """features."""
        return {
            "word": sentence[index],
            "is_first": index == 0,
            "is_last": index == len(sentence) - 1,
            # *ix
            "prefix-1": sentence[index][0],
            "prefix-2": sentence[index][:2],
            "prefix-3": sentence[index][:3],
            "suffix-1": sentence[index][-1],
            "suffix-2": sentence[index][-2:],
            "suffix-3": sentence[index][-3:],
            # word
            "prev_word": "" if index == 0 else sentence[index - 1],
            "two_prev_word": "" if index == 0 else sentence[index - 2],
            "next_word": "" if index == len(sentence) - 1 else sentence[index + 1],
            "two_next_word": (
                ""
                if index in {len(sentence) - 1, len(sentence) - 2}
                else sentence[index + 2]
            ),
            # digit
            "is_numeric": sentence[index].isdigit(),
            "prev_is_numeric": "" if index == 0 else sentence[index - 1].isdigit(),
            "next_is_numeric": (
                "" if index == len(sentence) - 1 else sentence[index + 1].isdigit()
            ),
            # punc
            "is_punc": self.__is_punc(sentence[index]),
            "prev_is_punc": "" if index == 0 else self.__is_punc(sentence[index - 1]),
            "next_is_punc": (
                ""
                if index == len(sentence) - 1
                else self.__is_punc(sentence[index + 1])
            ),
        }

    def tag(self: "POSTagger", tokens):
        """یک جمله را در قالب لیستی از توکن‌ها دریافت می‌کند و در خروجی لیستی از
        `(توکن، برچسب)`ها برمی‌گرداند.

        Examples:
            >>> posTagger = POSTagger(model = 'pos_tagger.model')
            >>> posTagger.tag(tokens = ['من', 'به', 'مدرسه', 'ایران', 'رفته_بودم', '.'])
            [('من', 'PRON'), ('به', 'ADP'), ('مدرسه', 'NOUN,EZ'), ('ایران', 'NOUN'), ('رفته_بودم', 'VERB'), ('.', 'PUNCT')]

            >>> posTagger = POSTagger(model = 'pos_tagger.model', universal_tag = True)
            >>> posTagger.tag(tokens = ['من', 'به', 'مدرسه', 'ایران', 'رفته_بودم', '.'])
            [('من', 'PRON'), ('به', 'ADP'), ('مدرسه', 'NOUN'), ('ایران', 'NOUN'), ('رفته_بودم', 'VERB'), ('.', 'PUNCT')]

        Args:
            tokens (List[str]): لیستی از توکن‌های یک جمله که باید برچسب‌گذاری شود.

        Returns:
            (List[Tuple[str,str]]): ‌لیستی از `(توکن، برچسب)`ها.

        """
        tagged_token = super().tag(tokens)
        return (
            self.__universal_converter(tagged_token)
            if self.__is_universal
            else tagged_token
        )

    def tag_sents(self: "POSTagger", sentences):
        """جملات را در قالب لیستی از توکن‌ها دریافت می‌کند
        و در خروجی، لیستی از لیستی از `(توکن، برچسب)`ها برمی‌گرداند.

        هر لیست از `(توکن، برچسب)`ها مربوط به یک جمله است.

        Examples:
            >>> posTagger = POSTagger(model = 'pos_tagger.model')
            >>> posTagger.tag_sents(sentences = [['من', 'به', 'مدرسه', 'ایران', 'رفته_بودم', '.']])
            [[('من', 'PRON'), ('به', 'ADP'), ('مدرسه', 'NOUN,EZ'), ('ایران', 'NOUN'), ('رفته_بودم', 'VERB'), ('.', 'PUNCT')]]

            >>> posTagger = POSTagger(model = 'pos_tagger.model', universal_tag = True)
            >>> posTagger.tag_sents(sentences = [['من', 'به', 'مدرسه', 'ایران', 'رفته_بودم', '.']])
            [[('من', 'PRON'), ('به', 'ADP'), ('مدرسه', 'NOUN'), ('ایران', 'NOUN'), ('رفته_بودم', 'VERB'), ('.', 'PUNCT')]]

        Args:
            sentences (List[List[str]]): لیستی از جملات که باید برچسب‌گذاری شود.

        Returns:
            (List[List[Tuple[str,str]]]): لیستی از لیستی از `(توکن، برچسب)`ها.
                    هر لیست از `(توکن،برچسب)`ها مربوط به یک جمله است.

        """
        tagged_sents = super().tag_sents(sentences)
        return (
            [self.__universal_converter(tagged_sent) for tagged_sent in tagged_sents]
            if self.__is_universal
            else tagged_sents
        )


class StanfordPOSTagger(stanford.StanfordPOSTagger):
    """StanfordPOSTagger."""

    def __init__(
        self: "StanfordPOSTagger",
        model_filename: "str",
        path_to_jar: str,
        *args, # noqa: ANN002
        **kwargs, # noqa: ANN003
    ) -> None:
        self._SEPARATOR = "/"
        super(stanford.StanfordPOSTagger, self).__init__(
            model_filename=model_filename,
            path_to_jar=path_to_jar,
            *args,  # noqa: B026
            **kwargs,
        )

    def tag(self: "StanfordPOSTagger", tokens):
        """tag.

        Examples:
            >>> tagger = StanfordPOSTagger(model_filename='persian.tagger', path_to_jar='stanford_postagger.jar')
            >>> tagger.tag(['من', 'به', 'مدرسه', 'رفته_بودم', '.'])
            [('من', 'PRO'), ('به', 'P'), ('مدرسه', 'N'), ('رفته_بودم', 'V'), ('.', 'PUNC')]

        """
        return self.tag_sents([tokens])[0]

    def tag_sents(self: "StanfordPOSTagger", sentences):
        """tag_sents."""
        refined = ([w.replace(" ", "_") for w in s] for s in sentences)
        return super(stanford.StanfordPOSTagger, self).tag_sents(refined)


class SpacyPOSTagger(POSTagger):
    def __init__(
        self: "SpacyPOSTagger",
        model_path=None,
        using_gpu=False,
        gpu_id=0
    ):
        """
        Initialize the SpacyPOSTagger with a model and data paths.

        Args:
        - model_path: Path to a pre-trained spaCy model.
        - test_dataset: Test dataset for evaluation. It has a similar structure to the training dataset.
        - test_directory: Directory to save the test data in spaCy format.
        - using_gpu: Set to True if you want use gpu (if you dont have one and set this to True the function use cpu automatically)
        This constructor calls the constructor of the parent class POSTagger.
        """
        super().__init__(universal_tag=True)
        self.model_path = model_path #### Usually an output directory for spacy model contain two other directory name model-last , model-best,You should give model_path like this : output/model-last
        self.using_gpu = using_gpu
        self.gpu_id = gpu_id
        self.tagger = None
        self._setup()

    def _setup(self: "SpacyPOSTagger"):
        """
        Set up the configuration for the spaCy model, including GPU settings and data preparation.

        This function initializes and configures the spaCy model, checks for GPU availability, and prepares the training and testing datasets in spaCy format.

        If using GPU, GPU settings are configured to enhance processing speed. Then, the pre-trained spaCy model is loaded based on the provided model path.

        Training and testing datasets are prepared and saved in the respective directories for use during model training and evaluation.
        """  # noqa: D212
        if self.using_gpu:
            self._setup_gpu()
        else:
            print("------------- You Prefer to use CPU --------------")


    def _setup_model(self: "SpacyPOSTagger",sents):
        """
        Initialize and configure the spaCy model for tagging without GPU settings.

        This method loads and configures the spaCy model based on the provided model path. It also sets up a custom tokenizer for text processing and constructs a dictionary for reference.

        Args:
            - model_path: Path to the pre-trained spaCy model.

        This method is typically called during setup to prepare the model for tagging tasks.
        """
        self.peykare_dict = {}  # Initialize a dictionary for reference
        self.tagger = spacy.load(self.model_path)  # Load the spaCy model
        self._set_peykare_dictionary(sents)  # Construct a reference dictionary
        self.tagger.tokenizer = self._custom_tokenizer  # Set a custom tokenizer for the model.

    def _setup_gpu(self: "SpacyPOSTagger"):
        """
        Check GPU availability and configure spaCy to use it if possible.

        This method checks whether a GPU is available and, if so, configures spaCy to utilize it for improved processing speed. It sets the 'gpu_availability' attribute to 'True' or 'False' accordingly.

        This check is performed during setup to make use of available GPU resources for enhanced performance.
        """
        print("------------------ GPU Setup Process Started ---------------------")
        gpu_available = spacy.prefer_gpu(gpu_id=self.gpu_id)  # Check if a GPU is available
        if gpu_available:
            print("------------ GPU is available and ready for use -------------")
            spacy.require_gpu(gpu_id=self.gpu_id)  # Configure spaCy to use the GPU
            self.gpu_availability = True
        else:
            print("------------ GPU is not available; spaCy will use CPU -------------")
            self.gpu_availability = False


    def _setup_dataset(self: "SpacyPOSTagger", dataset,saved_directory,data_type='train'):
        """
        Setup the training dataset in spaCy's binary format.

        This function prepares the training dataset and saves it in spaCy's binary format.
        """
        assert data_type in ['train','test']
        db = DocBin()
        for sent in tqdm(dataset):
            words = [word[0] for word in sent]
            tags = [word[1] for word in sent]
            doc = Doc(Vocab(strings=words), words=words)
            for d, tag in zip(doc, tags):
                d.tag_ = tag
            db.add(doc)

        self._handle_data_path(saved_directory)
        db.to_disk(f'{saved_directory}/{data_type}.spacy')

    def _handle_data_path(self,path='POSTaggerDataset'):
        """
        Create the directory if it doesn't exist.

        This method checks if the specified directory exists, and if not, it creates the directory to store the data.

        Args:
            - path: The path to the directory (default is 'POSTaggerDataset').

        This method is called to ensure the directory is available for saving processed data.
        """
        if not os.path.exists(path):
            os.makedirs(path)


    def _custom_tokenizer(self: "SpacyPOSTagger", text):
        """
        Implement a custom tokenization method for spaCy.

        This method defines a custom tokenization method for spaCy. It is used to tokenize input text based on a custom dictionary, or it raises an error if tokenization is not available.

        Args:
            - text: The input text to be tokenized.

        This custom tokenization method is used by the spaCy model during processing.

        """

        if text in self.peykare_dict:
            return Doc(self.tagger.vocab, self.peykare_dict[text])
        else:
            raise ValueError('No tokenization available for input.')

    def _set_peykare_dictionary(self: "SpacyPOSTagger", sents):
        """
        Create a dictionary for custom tokenization.

        This method constructs a dictionary to store custom tokenization mappings based on input sentences. It is used for custom tokenization in spaCy.

        Args:
            - sents: Input sentences to build the custom tokenization dictionary.

        This method is called during setup to establish a dictionary for tokenization.
        """
        self.peykare_dict = {' '.join([w for w in item]): [w for w in item] for item in sents}


    def _add_to_dict(self: "SpacyPOSTagger", sents):
        """
            Add the sentences to dictianory if it doesnt exist already
        """
        for sent in sents:
            key = ' '.join(sent)
            if key not in self.peykare_dict:
                self.peykare_dict[key] = sent


    def tag(self: "SpacyPOSTagger", tokens,universal_tag=True):
        """یک جمله را در قالب لیستی از توکن‌ها دریافت می‌کند و در خروجی لیستی از
        `(توکن، برچسب)`ها برمی‌گرداند.

        Examples:
            >>> posTagger = POSTagger(model = 'pos_tagger.model')
            >>> posTagger.tag(tokens = ['من', 'به', 'مدرسه', 'ایران', 'رفته_بودم', '.'])
            [('من', 'PRON'), ('به', 'ADP'), ('مدرسه', 'NOUN,EZ'), ('ایران', 'NOUN'), ('رفته_بودم', 'VERB'), ('.', 'PUNCT')]

            >>> posTagger = POSTagger(model = 'pos_tagger.model', universal_tag = True)
            >>> posTagger.tag(tokens = ['من', 'به', 'مدرسه', 'ایران', 'رفته_بودم', '.'])
            [('من', 'PRON'), ('به', 'ADP'), ('مدرسه', 'NOUN'), ('ایران', 'NOUN'), ('رفته_بودم', 'VERB'), ('.', 'PUNCT')]

        Args:
            tokens (List[str]): لیستی از توکن‌های یک جمله که باید برچسب‌گذاری شود.

        Returns:
            (List[Tuple[str,str]]): ‌لیستی از `(توکن، برچسب)`ها.

        """
        if self.tagger == None:
            self._setup_model([tokens])
        self._add_to_dict([tokens])

        text = ' '.join(tokens)
        doc = self.tagger(text)
        if not universal_tag:
            tags = [tok.tag_ for tok in doc]
        else:
            tags = [tok.tag_.replace(',EZ','') for tok in doc]

        return list(zip(tokens,tags))
              # noqa: W293

    def tag_sents(self:"SpacyPOSTagger",sents,universal_tag=True,batch_size=128):
        """
            Args:
                sents : List[List[Tokens]]
                batch_size : number of batches give to model for processing sentences each time
        """
        """
            Returns : List[List[Tuple(str,str)]]
        """
        if self.tagger == None:
            self._setup_model(sents)

        self._add_to_dict(sents)

        docs = list(self.tagger.pipe((' '.join([w for w in sent]) for sent in sents), batch_size=batch_size))
        if not universal_tag:
            tags = [[w.tag_ for w in doc] for doc in docs]
        else:
            tags = [[w.tag_.replace(',EZ','') for w in doc] for doc in docs]

        combined_out = [list(zip(tok,tag)) for tok,tag in zip(sents,tags)]
        return combined_out

    def train(
        self: "SpacyPOSTagger",
        train_dataset,
        test_dataset,
        data_directory,
        base_config_file,
        train_config_path,
        output_dir,
        use_direct_config=False
    ):
        """
        Train the spaCy model using a subprocess and a configuration file.

        This method executes the training process for the spaCy model by invoking spaCy's training module using subprocess. It takes input configuration files, training and testing data, and GPU settings (if available).

        Args:
            - train_dataset: Training dataset for the tagger. It is a list of sentences, where each sentence is a list of token-tag pairs.
            - test_dataset: Testing dataset for the tagger. It is a list of sentences, where each sentence is a list of token-tag pairs.
            - data_directory: Directory to save the training and testing data in spaCy format.
            - base_config_file: Path to the base configuration file for spaCy.
            - train_config_path: if use_direct_config set to True this is the path of config file for training that you will use
              if use_direct_config set to False this is the path that you want train config file will create with base_config
            - output_dir: Directory for storing the trained model and training logs.

        Upon successful training, this method updates the model path to the trained model.

        This method is typically called to initiate the training process of the spaCy model.
        """

        self.spacy_train_directory = data_directory
        self.train_dataset = train_dataset ### List[List[Tuple]]
        self.test_dataset = test_dataset
        if self.train_dataset:
            # Set up the training dataset configuration
            self._setup_dataset(dataset=self.train_dataset, saved_directory=self.spacy_train_directory, data_type='train')

        if self.test_dataset:
            self._setup_dataset(test_dataset,saved_directory=data_directory,data_type='test')

        train_data = f'{data_directory}/train.spacy'
        test_data = f'{data_directory}/test.spacy'

        if use_direct_config == False:
            self._setup_train_config(base_config_file, train_config_path=train_config_path)
        else:
            self.train_config_file = train_config_path

        command = f"python -m spacy train {self.train_config_file} --output ./{output_dir} --paths.train ./{train_data} --paths.dev ./{test_data}"
        if self.gpu_availability:
            command += f" --gpu-id {self.gpu_id}"

        subprocess.run(command, shell=True)
        self.model_path = f"{output_dir}/model-last"
        self._setup_model([[w for w,_ in sent] for sent in test_dataset])

    def _setup_train_config(self: "SpacyPOSTagger", base_config, train_config_path):
        """
        Create and configure the training configuration file for spaCy.

        This method sets up the training configuration file by copying a base configuration file and customizing it according to the specified parameters.

        Args:
            - base_config: Path to the base configuration file.
            - train_config_file_name: Name of the training configuration file for saving it.

        This method is called to generate the training configuration file used in the training process.
        """
        self.train_config_file = train_config_path
        print("----------------- Setting up the training configuration file ----------------------")
        command = f"python -m spacy init fill-config {base_config} {train_config_path}"  # Generate the training configuration file
        subprocess.run(command, shell=True)
        print("----------------- Training configuration file created successfully ----------------------")
        print(f"----------------- Training Config file address is {train_config_path} --------------------")

    def evaluate(self: "SpacyPOSTagger", test_sents,batch_size):
        """
        Evaluate the spaCy model on input sentences using different tag options.

        This method evaluates the spaCy model on input sentences with and without 'EZ' tags and reports classification metrics.

        Args:
            - sents: List of sentences for evaluation.
            - batch_size : number of batches that model should process each time
        This method calls the internal evaluation method for both tag options.

        This method is typically used for model evaluation and reporting metrics.
        """
        self._setup_model([[w for w,_ in sent] for sent in test_sents])
        if self.tagger:
            golds, predictions = self._get_labels_and_predictions(test_sents,batch_size)
            print("-----------------------------------------")
            self._evaluate_tags(test_sents, golds, predictions, use_ez_tags=True,batch_size=batch_size)
            print("-----------------------------------------")
            self._evaluate_tags(test_sents, golds, predictions, use_ez_tags=False,batch_size=batch_size)
        else:
            raise ValueError("Model does not exist.Please train a new one with train method of this instance or give a model_path , setup the model with self._setup_model() and then call evaluate")

    def _evaluate_tags(self, sents, golds=None, predictions=None, use_ez_tags=True,batch_size=128):
        """
        Evaluate model predictions and report classification metrics.

        This method evaluates model predictions and reports classification metrics for the specified tag option.

        Args:
            - sents: List of sentences for evaluation.
            - golds: List of gold labels (optional).
            - predictions: List of model predictions (optional).
            - use_ez_tags: A flag indicating whether to consider 'EZ' tags.
            - batch_size : number of batches model should process

        If `golds` and `predictions` are not provided, they are automatically extracted from the input sentences.

        This method calculates and displays precision, recall, and F1-score for the specified tag option.

        This method is called by the `evaluate` method to perform model evaluation.
        """
        if golds is None or predictions is None:
            golds, predictions = self._get_labels_and_predictions(sents,batch_size)

        predictions_cleaned = []
        golds_cleaned = []
        if use_ez_tags:
            get_tag_func = self._get_ez_tags
        else:
            get_tag_func = self._remove_ez_tags

        for preds, golds in zip(predictions, golds):
            for pred in preds:
                pred_cleaned = get_tag_func(pred)
                predictions_cleaned.append(pred_cleaned)
            for gold in golds:
                gold_cleaned = get_tag_func(gold)
                golds_cleaned.append(gold_cleaned)

        print(classification_report(golds_cleaned, predictions_cleaned))
        print('Precision: %.5f' % precision_score(golds_cleaned, predictions_cleaned, average='weighted'))
        print('Recall: %.5f' % recall_score(golds_cleaned, predictions_cleaned, average='weighted'))
        print('F1-Score: %.5f' % f1_score(golds_cleaned, predictions_cleaned, average='macro'))

    def _get_ez_tags(self, label):
        """
        Extract 'EZ' tags from labels.

        This method extracts 'EZ' tags from labels if they are present and returns them.

        Args:
            - label: The label containing 'EZ' tags.

        Returns:
            The 'EZ' tags or '-' if 'EZ' tags are not present.
        """
        if 'EZ' in label:
            label = 'EZ'
        else:
            label = '-'

        return label

    def _remove_ez_tags(self, label):
        """
        Remove 'EZ' tags from labels.

        This method removes 'EZ' tags from labels if they are present and returns the cleaned label.

        Args:
            - label: The label containing 'EZ' tags.

        Returns:
            The label with 'EZ' tags removed.
        """
        return label.replace(',EZ', '') if 'EZ' in label else label

    def _evaluate_ez_tags(self, sents):
        """
        Evaluate model predictions with 'EZ' tags included.

        This method evaluates model predictions with 'EZ' tags included.
        """
        self._evaluate_tags(sents, use_ez_tags=True)

    def _evaluate_normal_tags(self, sents):
        """
        Evaluate model predictions without 'EZ' tags.

        This method evaluates model predictions without 'EZ' tags.
        """
        self._evaluate_tags(sents, use_ez_tags=False)

    def _get_labels_and_predictions(self: "SpacyPOSTagger", sents,batch_size):
        """
        Extract gold labels and model predictions for evaluation.

        This method extracts gold labels and model predictions from input sentences.

        Args:
            - sents: List of sentences for evaluation.

        Returns:
            Lists of gold labels and model predictions.

        This method is typically used for gathering data to perform model evaluation.
        """
        gold_labels = [[tag for _, tag in sent] for sent in sents]
        tokens = [[w for w,_ in sent] for sent in sents]
        prediction_labels = self.tag_sents(tokens,batch_size)
        return gold_labels, prediction_labels


In [5]:
spacy_posTagger = SpacyPOSTagger(model_path = '/content/spacy_pos_tagger_parsbertpostagger', using_gpu=True)

------------------ GPU Setup Process Started ---------------------
------------ GPU is not available; spaCy will use CPU -------------


In [6]:
from hazm import WordTokenizer, Normalizer

tokenizer = WordTokenizer()
normalizer = Normalizer()

In [7]:
# Ezafe Correction
def get_last_subword(word):
    # Split the word by word boundaries
    subwords = re.findall(r'\b\w+\b', word)

    if len(subwords) > 1 and subwords[-1] in ['های', 'ی']:
      return (subwords[-2], subwords[-1])

    return (subwords[-1], '')

In [8]:
def get_EZ_tags(grapheme, compound=False):
  grapheme = re.sub('ۀ', 'ه‌ی', grapheme)
  tokens = tokenizer.tokenize(normalizer.normalize(grapheme))
  tags = spacy_posTagger.tag(tokens=tokens, universal_tag=False)
  tags = [(t[0], t[1], '') for t in tags if 'EZ' in t[1]]

  if compound:
    return tags

  splitted_tags = []
  for t in tags:
    subword1, subword2 = get_last_subword(t[0])
    splitted_tags.append((subword1, t[1], subword2))
  return splitted_tags

In [9]:
def get_naive_phonetic(word):
  char_map = {
  'ا': 'A', 'ب': 'b', 'پ': 'p', 'ت': 't', 'ث': 's', 'ج': 'j', 'چ': 'C',
  'ح': 'h', 'خ': 'x', 'د': 'd', 'ذ': 'z', 'ر': 'r', 'ز': 'z', 'ژ': 'Z',
  'س': 's', 'ش': 'S', 'ص': 's', 'ض': 'z', 'ط': 't', 'ظ': 'z', 'ع': '?',
  'غ': 'q', 'ف': 'f', 'ق': 'q', 'ک': 'k', 'گ': 'g', 'ل': 'l', 'م': 'm',
  'ن': 'n', 'و': 'v', 'ه': 'h', 'ی': 'y', 'ء': '?','ئ': '?', 'ؤ': '?',
  'آ': 'A', 'أ': '?', 'إ': '?', 'ۀ': 'eye'
  }
  mapped_string = ''.join(char_map.get(char, char) for char in word)
  return mapped_string

In [ ]:
import pandas as pd
merged_dict_path = 'final_merged_dict_pruned_POS_prob.csv'
merged_dict_df = pd.read_csv(merged_dict_path)

merged_dict = {}

for idx, row in merged_dict_df.iterrows():
  g, ps, nodiff, pos, prob = row['grapheme'], eval(row['phoneme']), row['nodiff'], eval(row['POS']), eval(row['prob'])
  if g not in merged_dict:
    merged_dict[g] = {'phoneme': [], 'nodiff': nodiff, 'POS': pos, 'prob': prob}

  for p in ps:
    merged_dict[g]['phoneme'].append(''.join(p))


inverted_merged_dict = {}

for key, value_list in merged_dict.items():
    for value in value_list['phoneme']:
        inverted_merged_dict[value] = key


def word_in_dict(word, inverted_dictionary=inverted_merged_dict):
  return word in inverted_dictionary

In [11]:
import itertools
def get_word_phonetic_candidates(word):
  subwords = []
  for match in re.finditer(r'\b(\w+)\b', word):
      match_text = match.group()
      match_span = match.span()
      subwords.append((match_text, match_span))

  subword_candidates = []
  for subword, _ in subwords:
    if subword in merged_dict:
      subword_candidates.append(merged_dict[subword]['phoneme'])
    else:
      subword_candidates.append([get_naive_phonetic(subword)])

  # Generate all possible combinations and concatenate
  word_candidates = [''.join(comb) for comb in itertools.product(*subword_candidates)]
  return word_candidates

In [12]:
def get_updated_span(match_span, displacements):
  new_start, new_end = match_span[0], match_span[1]
  for start, displacement in displacements:
    if start <= new_start:
      new_start += displacement
      new_end += displacement

  return (new_start, new_end)

In [13]:
def add_1_for_EZ(phonemes):
    # Convert 'a' to 'A' in '-ha' and '-haye'
    phonemes = phonemes.replace('-ha', '-hA').replace('-haye', '-hAye')
    
    # Add '1' after '-e', '-ye', and '-hAye'
    phonemes = phonemes.replace('-e', '-e1').replace('-ye', '-ye1').replace('-hAye', '-hAye1')
    
    return phonemes

In [14]:
def correct_phonetic_model_EZ_by_tags(grapheme, model_output):
  EZ_tags = get_EZ_tags(grapheme, compound=False)

  matches = []
  matched_spans = set()

  for word, tag, ending in EZ_tags:
    phonetic_candidates = get_word_phonetic_candidates(word)

    for phonetic in phonetic_candidates:
      if phonetic.endswith('Aye') and not word.endswith('ه'): continue
      for match in re.finditer(r'\b(\w+)\b', model_output):
        match_text = match.group()
        match_span = match.span()
        
        if match_text not in ['e', 'ye', 'i', 'ha', 'hA', 'hAye', 'haye'] and match_span not in matched_spans and SequenceMatcher(None, match_text, phonetic).ratio() > 0.75:
          matches.append((match_span, match_text, phonetic, ending))
          matched_spans.add(match_span)

  non_matches = []
  for match in re.finditer(r'\b(\w+)\b', model_output):
    match_text = match.group()
    match_span = match.span()
    if not match_text.endswith('e') and not match_text.endswith('ye') and not match_text.endswith('i') and not match_text.endswith('ha')and not match_text.endswith('hAye') and not match_text.endswith('haye') and match_span not in matched_spans:
      non_matches.append((match_text, match_span))

  displacements = []
  for match_span, _, phonetic, ending in matches:
    match_span = get_updated_span(match_span, displacements)
    if model_output[match_span[1]:].startswith('-e') or model_output[match_span[1]:].startswith('-ye') or model_output[match_span[1]:].startswith('-hAye') or model_output[match_span[1]:].startswith('-haye') or model_output[match_span[1]:].startswith('-hA-ye') or model_output[match_span[1]:].startswith('-ha-ye'):
      continue

    output_word = model_output[match_span[0]:match_span[1]]
    if len(output_word) >= 4 and output_word[-3] in 'еeiuoaāäâāɒáA' and output_word.endswith('ye') and \
     ((word_in_dict(output_word[:-2], inverted_merged_dict) and not word_in_dict(output_word, inverted_merged_dict)) or \
     SequenceMatcher(None, output_word[:-2], phonetic).ratio() > SequenceMatcher(None, output_word, phonetic).ratio()):
      model_output = model_output[:match_span[1] - 2] + '-' + model_output[match_span[1] - 2:]
      displacements.append((match_span[1] - 2, 1))
      continue

    if len(output_word) >= 3 and output_word.endswith('e') and \
     ((word_in_dict(output_word[:-1], inverted_merged_dict) and not word_in_dict(output_word, inverted_merged_dict)) or \
     SequenceMatcher(None, output_word[:-1], phonetic).ratio() > SequenceMatcher(None, output_word, phonetic).ratio()):
      model_output = model_output[:match_span[1] - 1] + '-' + model_output[match_span[1] - 1:]
      displacements.append((match_span[1] - 1, 1))
      continue

    if ending == 'ی' and len(output_word) >= 4 and output_word[-2:] == 'ye' and output_word[-3] == phonetic[-1]:
      model_output = model_output[:match_span[1] - 2] + '-' + model_output[match_span[1] - 2:]
      displacements.append((match_span[1] - 2, 1))
      continue

    if ending == 'های' and len(output_word) >= 7 and output_word[-5:] == 'hAye':
      model_output = model_output[:match_span[1] - 4] + '-' + model_output[match_span[1] - 4:]
      displacements.append((match_span[1] - 5, 1))
      continue

    if ending == 'های' and len(output_word) >= 6 and output_word[-4:] == 'haye':
      model_output = model_output[:match_span[1] - 4] + '-hAye' + model_output[match_span[1]:]
      displacements.append((match_span[1] - 4, 1))
      continue

    if ending not in ['ی', 'های'] and len(output_word) >= 3 and output_word[-2] == phonetic[-1] and output_word[-1] == 'e':
      model_output = model_output[:match_span[1] - 1] + '-' + model_output[match_span[1] - 1:]
      displacements.append((match_span[1] - 1, 1))
      continue

    if output_word[-1] in 'еeiuoaāäâāɒáA' and not output_word.endswith('haye') and not output_word.endswith('hAye'):
      model_output = model_output[:match_span[1]] + '-ye' + model_output[match_span[1]:]
      displacements.append((match_span[1], 3))
      continue

    if not output_word.endswith('e'):
      model_output = model_output[:match_span[1]] + '-e' + model_output[match_span[1]:]
      displacements.append((match_span[1], 2))

  for non_match, match_span in non_matches:
    match_span = get_updated_span(match_span, displacements)
    output_word = model_output[match_span[0]:match_span[1]]
    if re.match(r'^-e\b', model_output[match_span[1]:]):
      model_output = model_output[:match_span[1]] + model_output[match_span[1] + 2:]
      displacements.append((match_span[1] + 2, -2))
      continue

    if re.match(r'^-ye\b', model_output[match_span[1]:]):
      model_output = model_output[:match_span[1]] + model_output[match_span[1] + 3:]
      displacements.append((match_span[1] + 3, -3))
      continue

    if len(output_word) >= 4 and output_word[-3] in 'еeiuoaāäâāɒáA' and output_word.endswith('ye') and  (word_in_dict(output_word[:-2], inverted_merged_dict) and not word_in_dict(output_word, inverted_merged_dict)):
      model_output = model_output[:match_span[1] - 2] + model_output[match_span[1]:]
      displacements.append((match_span[1], -2))
      continue

    if len(output_word) >= 3 and output_word.endswith('e') and (word_in_dict(output_word[:-1], inverted_merged_dict) and not word_in_dict(output_word, inverted_merged_dict)):
      model_output = model_output[:match_span[1] - 1] + model_output[match_span[1]:]
      displacements.append((match_span[1], -1))


  model_output = add_1_for_EZ(model_output)

  return model_output

# Get Data

## Get Merged Dict

In [ ]:
merged_dict_path = "final_merged_dict_pruned_POS_prob.csv"

In [16]:
import pandas as pd
merged_dict_df = pd.read_csv(merged_dict_path)

In [17]:
merged_dict_df.shape

(116630, 5)

In [18]:
consonants_regex = '(?=' + '|'.join(['q', 'r', 't', 'y', 'p', 's', 'd', 'f', 'g', 'h', 'j', 'k', 'l', 'z', 'x', 'c', 'v', 'b', 'n', 'm', 'Q', 'R', 'T', 'Y', 'P', 'S', 'D', 'F', 'G', 'H', 'J', 'K', 'L', 'Z', 'X', 'C', 'V', 'B', 'N', 'M' ]) + ')'
vowels_regex = '(?=' + '|'.join(['a', 'A', 'e', 'i', 'u', 'o']) + ')'

In [19]:
merged_dict = {}

for idx, row in merged_dict_df.iterrows():
  g, ps, nodiff, pos, prob = row['grapheme'], eval(row['phoneme']), row['nodiff'], eval(row['POS']), eval(row['prob'])
  if g not in merged_dict:
    merged_dict[g] = {'phoneme': [], 'nodiff': nodiff, 'POS': pos, 'prob': prob}

  for p in ps:
    merged_dict[g]['phoneme'].append(''.join(p))

In [23]:
inverted_merged_dict = {}

for key, value_list in merged_dict.items():
    for value in value_list:
        inverted_merged_dict[value] = key

In [24]:
def word_in_dict(word, inverted_dictionary=inverted_merged_dict):
  return word in inverted_dictionary

## Get Evaluation Data

In [ ]:
sentence_bench = pd.read_csv('SentenceBench.csv')

In [26]:
sentence_bench.head(3)

,dataset,grapheme,phoneme,homograph word,pronunciation
0,polyphone,من قدر تو را می‌دانم,man qadr-e to rA mi-dAnam,قدر,qadr
1,polyphone,از قضای الهی به قدر الهی پناه می‌برم,?az qazAy ?elAhi be qadar-e ?elAhi panAh mi-baram,قدر,qadar
2,polyphone,به دست و صورتم کرم زدم,be dast-o suratam kerem zadam,کرم,kerem


### Get ManaTTS

In [27]:
filtered_rows = sentence_bench[sentence_bench['dataset'] == 'mana-tts'][['grapheme', 'phoneme']]

# Convert to a list of tuples
mana_evaluation_data = list(filtered_rows.itertuples(index=False, name=None))

mana_evaluation_data[:3]

[('در این نوشته بنا داریم با یک ابزار ساده و مکانیکی افزایش بینایی برای افراد کم\u200cبینا ',
  'dar ?in neveSte banA dArim bA yek ?abzAr-e sAde va mekAniki-ye ?afzAyeS-e binAyi barAye ?afrAd-e kam\u200cbinA '),
 ('به نام بی\u200cوپتیک یا عدسی دورنما آشنا شویم. ',
  'be nAm-e biyoptik yA ?adasi-ye durnamA ?ASnA Savim'),
 ('دراین\u200cصورت، انجام خودارزیابی و ارائه بازخورد بر عهده خودتان است. ',
  'dar ?in surat ?anjAm-e xod?arzyAbi va ?erA?e-ye bAzxord bar ?ohde-ye xodetAn ?ast ')]

### Get CommonVoice

In [28]:
filtered_rows = sentence_bench[sentence_bench['dataset'] == 'commonvoice'][['grapheme', 'phoneme']]

# Convert to a list of tuples
commonvoice_evaluation_data = list(filtered_rows.itertuples(index=False, name=None))

commonvoice_evaluation_data[:3]

[('در اکثر شهرها، مرکزی برای خرید دوچرخه وجود دارد.',
  'dar ?aksar-e Sahr-hA, markazi barAye xarid-e  doCarxe vojud dArad.'),
 ('پس از مدرسه کودکان به سوی خانه جست و خیز کردند.',
  'pas ?az madrese kudakAn be suye xAne jast-o-xiz kardand.'),
 ('شما نگران زن و بچه این نباش.', 'SomA negarAn-e zan-o-baCCe-ye ?in nabAS.')]

### Get Ambiguious

In [29]:
filtered_rows = sentence_bench[sentence_bench['dataset'] == 'polyphone'][['grapheme', 'phoneme', 'homograph word',	'pronunciation']]

# Convert to a list of tuples
ambiguous_evaluation_data = list(filtered_rows.itertuples(index=False, name=None))

ambiguous_evaluation_data[:3]

[('من قدر تو را می\u200cدانم', 'man qadr-e to rA mi-dAnam', 'قدر', 'qadr'),
 ('از قضای الهی به قدر الهی پناه می\u200cبرم',
  '?az qazAy ?elAhi be qadar-e ?elAhi panAh mi-baram',
  'قدر',
  'qadar'),
 ('به دست و صورتم کرم زدم', 'be dast-o suratam kerem zadam', 'کرم', 'kerem')]

# Evaluate Method Outputs

## PER Evaluation

In [30]:
def remove_non_word_chars(text):
    pattern = r'[^\w\s\?]'
    cleaned_text = re.sub(pattern, ' ', text)
    return cleaned_text

In [31]:
def remove_white_spaces(text):
    cleaned_text = re.sub(r'\s+', ' ', text)
    return cleaned_text.strip()

In [32]:
def get_word_only_text(text):
  word_only_text = remove_non_word_chars(text)
  extra_space_removed_text = remove_white_spaces(word_only_text)

  return extra_space_removed_text

In [33]:
def get_texts_cer(reference, model_output):
  # Preprocess input texts to only contain word characters
  word_only_reference = get_word_only_text(reference)
  word_only_output = get_word_only_text(model_output)

  # Return +infinity for CER if any of the texts is empty
  if not word_only_reference.strip() or not word_only_output.strip():
    return float('inf')

  return cer(word_only_reference, word_only_output)

In [34]:
def get_avg_cer_of_method(method_outputs, references):
  cers = []
  for idx, o in enumerate(method_outputs):
    cer = get_texts_cer(o, references[idx][1].replace('-', ''))
    if cer != float('inf'):
      cers.append(cer)

  return sum(cers) / len(cers)

## Ezafe Evaluation

In [35]:
def get_EZ_words_from_ground_truth(text):
  pattern = r'\b(\w+)(-e|-ye)\b'
  matches = re.findall(pattern, text,)

  # Extract the words along with the suffix
  words_with_suffix = [match[0] + match[1] for match in matches]
  EZ_words = [tuple(re.split(r'(?=-)', w)) for w in words_with_suffix]

  return EZ_words

In [36]:
def get_EZ_words_from_phonetic_model_output(text):
    EZ_words = re.findall(r'\b(\w+)(-e|-ye)', text)
    EZ_word_candidates = []

    other_words = re.findall(r'\b(\w+)(?=(?:[^-\w]|$))', text)
    for word in other_words:
      if len(word) >= 4 and word[-3] in 'еeiuoaāäâāɒáA' and word.endswith('ye') and word_in_dict(word[:-2], inverted_merged_dict) and not word_in_dict(word, inverted_merged_dict) and not word_in_dict(word[:-1], inverted_merged_dict):
        EZ_words.append((word[:-2], '-ye'))
        continue

      if len(word) >= 3 and word.endswith('e') and word_in_dict(word[:-1], inverted_merged_dict) and not word_in_dict(word, inverted_merged_dict):
        EZ_words.append((word[:-1], '-e'))
        continue

      if len(word) >= 4 and word[-3] in 'еeiuoaāäâāɒáA' and word.endswith('ye'):
        EZ_word_candidates.append((word[:-2], '-ye'))
        continue

      if len(word) >= 3 and word.endswith('e'):
        EZ_word_candidates.append((word[:-1], '-e'))

    return EZ_words, EZ_word_candidates

In [37]:
def get_phonetic_model_TP_FP_TN_FN(gt_finglish, model_finglish):
  gt_word_count = len(re.findall(r'\b\w+(?:-\w+)*\b', gt_finglish))
  gt_EZ_words = get_EZ_words_from_ground_truth(gt_finglish)

  model_EZ_words, model_candidate_EZ_words = get_EZ_words_from_phonetic_model_output(model_finglish)

  TP = 0
  FP = 0
  TN = 0
  FN = 0

  gt_matched_indices = set()
  model_matched_indices = set()
  model_candidate_matched_indices = set()

  for gt_idx, (word, EZ) in enumerate(gt_EZ_words):
    for model_idx, (w, E) in enumerate(model_EZ_words):
      if model_idx not in model_matched_indices and SequenceMatcher(None, word, w).ratio() > 0.65:
        TP += 1
        gt_matched_indices.add(gt_idx)
        model_matched_indices.add(model_idx)
        break
    else:
      for model_c_idx, (w, E) in enumerate(model_candidate_EZ_words):
        if model_c_idx not in model_candidate_matched_indices and SequenceMatcher(None, word, w).ratio() > 0.65:
          TP += 1
          gt_matched_indices.add(gt_idx)
          model_candidate_matched_indices.add(model_c_idx)
          break

  # Calculate FP: model_EZ_words that are not TP
  FP = len(model_EZ_words) - (TP - len(list(model_candidate_matched_indices)))

  # Calculate FN: gt_EZ_words that were not detected
  FN = len(gt_EZ_words) - TP

  # Calculate TN: non-Ezafe words that are correctly not detected as Ezafe
  TN = (gt_word_count - len(gt_EZ_words)) - FP

  return TP, FP, TN, FN


In [38]:
def get_phonetic_model_performance(outputs, references):
  total_TP, total_FP, total_TN, total_FN = 0, 0, 0, 0

  for idx, o in enumerate(outputs):
    TP, FP, TN, FN = get_phonetic_model_TP_FP_TN_FN(references[idx][1], o)
    total_TP += TP
    total_FP += FP
    total_TN += TN
    total_FN += FN


  total_model_EZ = total_TP + total_FP
  total_gt_EZ = total_TP + total_FN

  total_model_T = total_TP + total_TN

  total_gt_words = total_TP + total_TN + total_FP + total_FN

  accuracy = (total_model_T) / (total_gt_words) * 100
  precision = (total_TP) / (total_model_EZ) * 100
  recall = (total_TP) / (total_gt_EZ) * 100

  return accuracy, precision, recall

## Polyphone Evaluation

In [39]:
def get_polyphone_performance(outputs, references):
  corrects = 0
  total = 0

  for idx, (g, p, polyphone, right) in enumerate(references):
    if polyphone != '':
      total += 1
      if right in outputs[idx]:
        corrects += 1
      else:
        print(f"Out: {outputs[idx]}\nRef: {p}")

  return corrects / total

# Full bench

In [40]:
benchmark = []

for g, p in mana_evaluation_data:
  benchmark.append((g, p, '', ''))

for g, p in commonvoice_evaluation_data:
  benchmark.append((g, p, '', ''))

for g, p, w, r in ambiguous_evaluation_data:
  benchmark.append((g, p, w, r))

benchmark = benchmark[:400]

In [41]:
def print_all_metrics(predictions):
  per = get_avg_cer_of_method(predictions, benchmark) * 100
  acc, prec, recall = get_phonetic_model_performance(predictions, benchmark)
  polyphone = get_polyphone_performance(predictions, benchmark) * 100

  print(f"PER: \t\t\t{per:.2f}")
  print(f"ACC, PREC, RECALL, F1: \t{acc:.2f}, {prec:.2f}, {recall:.2f}, {((2 * prec * recall) / (prec + recall)):.2f}")
  print(f"POLYPHONE: \t\t{polyphone:.2f}")

# Inference

In [42]:
graphemes = [item[0] for item in benchmark]

In [43]:
outputs = g2p.generate(graphemes, use_rules=True)

# Mapping

In [44]:
print(set(''.join(outputs)))

{'k', '$', '/', 'l', 'f', '1', 'g', ';', 'e', 'x', ' ', 'z', 'o', '@', 'a', 'b', 's', 'p', 'h', 'd', 'j', 'c', 'v', 'r', 'm', 'n', 'y', 't', 'u', 'i', 'q'}


In [45]:
# Define the replacements
replacements = {
    'a': 'A',
    '$': 'S',
    '/': 'a',
    '1': '',
    ';': 'Z',
    '@': '?',
    'c': 'C'
}

# Apply replacements
mapped_outputs = [
    ''.join(replacements.get(char, char) for char in output)
    for output in outputs
]

# Results

In [46]:
print_all_metrics(mapped_outputs)

Out: ?az qazAye ?elAhi beqadr ?elAhi panAh mibaram
Ref: ?az qazAy ?elAhi be qadar-e ?elAhi panAh mi-baram
Out: ?agar be Sahre maro safar koni
Ref: ?agar be Sahr-e marv safar koni
Out: ?az piSe man mero nemitavAnam bA to biyAyam
Ref: ?az piS-e man maro nemi-tavAnam bA to biyAyam
Out: zendegi tabaqe xAsteye digarAn saxt ?ast
Ref: zendegi tebq-e xAste-ye digarAn saxt ?ast
Out: jazr ?o madde daryA padideye ?ajibi ?ast
Ref: jazr va madd-e daryA padide-ye ?ajibi ?ast
Out: mas?olin dar har samt va pasti ke bASand bAyad xedmat konand
Ref: mas?ulin dar har semat va posti ke bASand bAyad xedmat konand
Out: ?engadr zar nazan xaste Sodim
Ref: ?enqadr zer nazan xaste Sodim
Out: qadri darham ?o dinAr nasibaS Sod
Ref: qadri derham va dinAr nasibaS Sod
Out: ?ey meh zibAye man ?AftAb rA xajal kardei
Ref: ?ey mah-e zibAy-e man ?AftAb rA xejel karde-?i
Out: Savi ?u ruze ba?d ?az ?ezdevAj rahlat namud
Ref: Suye ?u ruz-e ba?d ?az ?ezdevAj rehlat nemud
Out: ?agar xoSAl Savi bAyad Sokr begozAri
Ref: ?agar xo